In [2]:
from tool import *
from tqdm import tqdm

In [3]:
# general options
pd.set_option('display.max_columns',100)
pd.set_option('display.max_rows',60)
pd.set_option('display.max_colwidth', None)

## 1. 양품 S/N 자료를 전수검사 DB에 업데이트

In [89]:
df1 = pd.read_excel(get_filename(), skiprows=2, sheet_name='SN') # QA 전수검사 엑셀파일의 양품S/N 탭 자료 불러옴, 상단에서 2줄 Skip함
df1.columns = df1.columns.astype(str) # 컬럼명을 String으로 변경

#  상기 데이타프레임의 컬럼명에서 날짜만 추출한 것으로 새로운 컬럼명 생성
new_col = [i for i in df1.columns if i.startswith('Unn') != True]
new_col = [i for i in new_col if i.startswith('날') != True]

df1 = df1[new_col] # 새로운 컬럼명으로 데이타프레임 변경

new_col = [] # 새로운 컬럼명에서 날짜명을 잘라서 컬럼명 재변경
for col in df1.columns:
    new_col.append(col[:10])
df1.columns = new_col

# 비어있는 데이타프레임 생성 후 거기에 위에서 만든 데이타프레임의 정보를 추출하여 가공하여 새로운 포맷으로 만듬
df_new = pd.DataFrame()
for num, col in enumerate(df1.columns):
    for sn in df1.iloc[:,num].dropna()[2:]: # 3번째 S/N 데이타부터 가져오기 위해 slicing
        temp_dic = {'Inspection Date':col, 'Mapping Model.Suffix':df1.iloc[:, num][0], 'S/N':sn}
        df_new = df_new.append(temp_dic, ignore_index=True)
df_new['Judge'] = 'OK'
df_new['Inspection Date'] = pd.to_datetime(df_new['Inspection Date'])

### 전수검사 사유 및 책임 소재 기입

In [92]:
df_new

,Inspection Date,Mapping Model.Suffix,S/N,Judge
0,2023-03-30,17UD70Q-P.AX70K,303PGKW815765,OK
1,2023-03-30,17UD70Q-P.AX70K,303PGYZ815754,OK
2,2023-03-30,17UD70Q-P.AX70K,303PGKW815693,OK
3,2023-03-30,17UD70Q-P.AX70K,303PGSY815710,OK
4,2023-03-30,17UD70Q-P.AX70K,303PGHT815897,OK
...,...,...,...,...
410,2023-03-30,17UD70Q-P.AX70K,303PGWR815889,OK
411,2023-03-30,17UD70Q-P.AX70K,303PGEF815956,OK
412,2023-03-30,17UD70Q-P.AX70K,303PGJK815892,OK
413,2023-03-30,17UD70Q-P.AX70K,303PGMP815859,OK


In [93]:
# 전수검사 사유의 예
reason1 = 'Full inspection for quality control'
reason2 = 'Full inspection for sampling inspection NG( 16T90R : Caps Lock Key Crack, C-part stain, Seal Label peel off)'
reason3 = 'Full inspection for sampling inspection NG(16T90Q LCM noise)'
reason4 = 'Full inspection for sampling inspection NG(Noise problem caused by internal screw influx in the product)'
reason5 = 'Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))'

reason = reason5

df_new['Responsibility'] = 'Supplier'
df_new['Inspection_Reason'] = reason

In [94]:
df_new

,Inspection Date,Mapping Model.Suffix,S/N,Judge,Responsibility,Inspection_Reason
0,2023-03-30,17UD70Q-P.AX70K,303PGKW815765,OK,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
1,2023-03-30,17UD70Q-P.AX70K,303PGYZ815754,OK,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
2,2023-03-30,17UD70Q-P.AX70K,303PGKW815693,OK,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
3,2023-03-30,17UD70Q-P.AX70K,303PGSY815710,OK,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
4,2023-03-30,17UD70Q-P.AX70K,303PGHT815897,OK,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
...,...,...,...,...,...,...
410,2023-03-30,17UD70Q-P.AX70K,303PGWR815889,OK,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
411,2023-03-30,17UD70Q-P.AX70K,303PGEF815956,OK,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
412,2023-03-30,17UD70Q-P.AX70K,303PGJK815892,OK,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
413,2023-03-30,17UD70Q-P.AX70K,303PGMP815859,OK,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"


In [95]:
df_new['S/N'].str.len().value_counts() # S/N 자리수가 13자리 맞는지 확인

13    415
Name: S/N, dtype: int64

In [96]:
df_new[~df_new['Mapping Model.Suffix'].str.contains(odm_model)] # OS모델명 형태와 맞지 않는 모델명 있는지 확인

,Inspection Date,Mapping Model.Suffix,S/N,Judge,Responsibility,Inspection_Reason


In [46]:
with open('D:/Data/PC Inspection DB.bin', 'rb') as f:
    df = pickle.load(f)

In [98]:
df.shape

(108568, 12)

In [99]:
sample = pd.DataFrame() # DB에 update되는 신규 데이타만 취합하기 위해 만든 빈 dataframe
for row in tqdm(df_new.iterrows()):
    cond = (df['Inspection Date'] == row[1]['Inspection Date']) & (df['S/N'] == row[1]['S/N']) & (df['Judge'] == row[1]['Judge']) & (df['Mapping Model.Suffix'] == row[1]['Mapping Model.Suffix'])
    if df[cond].shape[0] == 0:
        df = df.append(row[1])
        sample = sample.append(row[1])
df.reset_index(drop=True, inplace=True)

415it [00:34, 11.95it/s]


In [100]:
sample.shape # 신규로 추가된 S/N 개수 확인

(415, 6)

In [101]:
df.shape

(108983, 12)

In [102]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

In [103]:
with open('D:/Data/DB backup/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

## 2. 불량 탭에서 데이타 가져와서 불량 내역을 영어로 번역하기

In [104]:
df1 = pd.read_excel(get_filename(), skiprows=2, sheet_name='불량현황') # 특정 불량 모델의 탭의 자료 가져오기

In [105]:
df1 = df1.loc[:, df1.columns[df1.columns.str.contains('검사')][0]:]
df1.columns = ['Inspection Date', 'Mapping Model.Suffix', 'MFG DATE', 'S/N', 'Defect Reason_Korean', ''] # 컬럼명 변경

In [106]:
df1 = df1.dropna(how='all') # 전체 행에 빈값이 있는것은 다 날림

In [107]:
df1 = df1.drop('', axis=1) # 불필요한 컬럼 삭제

In [108]:
df1.insert(4, 'Judge', 'NG') # judge 컬럼 추가하여 NG로 표시

In [110]:
df1['Responsibility'] = 'Supplier'
df1['Inspection_Reason'] = reason

In [111]:
df1 = df1[df1['S/N'].notnull()]

In [112]:
df1['Defect Reason_Korean'] = df1['Defect Reason_Korean'].str.replace('\n', ',')

In [113]:
df1 # 취합 데이터 확인

,Inspection Date,Mapping Model.Suffix,MFG DATE,S/N,Judge,Defect Reason_Korean,Responsibility,Inspection_Reason
0,2023-03-30,17UD70Q-P.AX70K,2023-03-07,72608303PGUH815814,NG,"평탄도 불량,좌 0.8mm",Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
1,2023-03-30,17UD70Q-P.AX70K,2023-03-07,72608303PGTV815689,NG,"평탄도 불량,좌 0.6mm",Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
2,2023-03-30,17UD70Q-P.AX70K,2023-03-06,72608303PGJK815916,NG,"평탄도 불량,좌 1.0mm",Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
3,2023-03-30,17UD70Q-P.AX70K,2023-03-06,72608303PGCD815827,NG,"평탄도 불량,좌 1.0mm",Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
4,2023-03-30,17UD70Q-P.AX70K,2023-03-06,72608303PGHT815585,NG,"평탄도 불량,좌 1.1mm",Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
5,2023-03-30,17UD70Q-P.AX70K,2023-03-04,72608303PGMP815907,NG,D-Part 얼룩,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
6,2023-03-30,17UD70Q-P.AX70K,2023-03-06,72608303PGXA815938,NG,Box 불량,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
7,2023-03-30,17UD70Q-P.AX70K,2023-03-07,72608303PGRS815774,NG,"평탄도 불량,좌 0.7mm",Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
8,2023-03-30,17UD70Q-P.AX70K,2023-03-06,72608303PGKW815621,NG,LCD Open / Close 소음,Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"
9,2023-03-30,17UD70Q-P.AX70K,2023-03-07,72608303PGFJ816011,NG,"평탄도 불량,좌 1.2mm",Supplier,"Full inspection for sampling inspection NG(Poor flatness (Spec: max 0.5mm, Actual: over 0.8mm))"


### 1) 번역 (by Google cloud API)

In [114]:
df1['Defect Reason_English'] = df1['Defect Reason_Korean'].apply(get_google_translate, args=['en']) # Google cloud API 월 번역용량 제한있음

In [115]:
df1['Defect Reason_English'] = df1['Defect Reason_English'].str.replace('&quot;', '"')
df1['Defect Reason_English'] = df1['Defect Reason_English'].str.replace('&#39;', '"')

In [116]:
df1[['Defect Reason_Korean', 'Defect Reason_English']][:] # 번역 결과 확인

,Defect Reason_Korean,Defect Reason_English
0,"평탄도 불량,좌 0.8mm","Poor flatness, left 0.8mm"
1,"평탄도 불량,좌 0.6mm","Poor flatness, left 0.6mm"
2,"평탄도 불량,좌 1.0mm","Poor flatness, left 1.0mm"
3,"평탄도 불량,좌 1.0mm","Poor flatness, left 1.0mm"
4,"평탄도 불량,좌 1.1mm","Poor flatness, left 1.1mm"
5,D-Part 얼룩,D-Part stain
6,Box 불량,Box bad
7,"평탄도 불량,좌 0.7mm","Poor flatness, left 0.7mm"
8,LCD Open / Close 소음,LCD Open / Close Noise
9,"평탄도 불량,좌 1.2mm","Poor flatness, left 1.2mm"


### 번역 수정할 때

In [46]:
keyword = 'Key 무감'

In [47]:
df1.loc[df1['Defect Reason_Korean'].str.contains(keyword), 'Defect Reason_English']

1    No key, (no key including F2 key)
Name: Defect Reason_English, dtype: object

In [48]:
df1.loc[df1['Defect Reason_Korean'].str.contains(keyword), 'Defect Reason_English'] = 'Many Key insensitivity(Including F2 Key)'

In [49]:
df1

,Inspection Date,Mapping Model.Suffix,MFG DATE,S/N,Judge,Defect Reason_Korean,Responsibility,Inspection_Reason,Defect Reason_English
0,2023-02-28,16TD90R-G.AX76K,2023-02-24,302QCUK564022,NG,LCD 흑점,LG,Full inspection for quality control,LCD black dot
1,2023-02-28,16TD90R-G.AX56K,2023-02-21,302QCTB561977,NG,"Key 무감,(F2 Key 포함 다수 Key 무감)",LG,Full inspection for quality control,Many Key insensitivity(Including F2 Key)
2,2023-02-28,16TD90R-G.AX56K,2023-02-17,302QCUK561934,NG,"D-Part 누름시 소음,(꿀렁꿀렁됨)",LG,Full inspection for quality control,Noise when pressing D-Part


### 2) 번역 (by 구글 번역 Site using 셀레늄)

In [57]:
# 셀레늄 이용하므로 속도 느리나, 한개씩 번역할 때 번역 정확도 높은 편
df1['Defect Reason_English'] = df1['Defect Reason_Korean'].apply(google_translate)

In [58]:
df1 # 번역결과 확인

,Inspection Date,Mapping Model.Suffix,MFG DATE,S/N,Judge,Defect Reason_Korean,Defect Reason_English
0,2022-03-03,17U70P-P.AA7SK,2022-01-26,201PGPM654448,NG,Image 설치 실패 (Error 창 발생),Image installation failure (Error window occurs)
1,2022-03-03,17U70P-P.AA7SK,2022-01-26,201PGBL654397,NG,Image 설치 실패 (Error 창 발생),Image installation failure (Error window occurs)


### 3) 번역 (by 구글 번역 Site using 셀레늄 with large string data)

In [151]:
# 번역해야할 양이 많을 때, 구글 번역하기 위한 불량내용(한국어)를 1개의 text로 만듬.
total_text = ''
for text in df1['Defect Reason_Korean']:
    text = text.replace('\n', ' ')
    total_text = total_text + text + '\n'

In [152]:
len(total_text.split('\n')[:-1]) # 전체 번역할 text의 한줄씩 전체 개수

102

In [153]:
result = google_translate(total_text) # 셀레늄으로 구글 번역한 결과 저장

In [154]:
len(result.split('\n')) # 결과 text 를 한줄씩 전체 개수 확인(위의 개수와 맞는지 확인)

102

In [155]:
df.loc[:, 'Defect Reason_English'] = result.split('\n') # 맞으면, 새로운 불량내용(영어) 컬럼에 번역내용 붙여넣기

In [31]:
df # 결과 확인

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,RMA Number,Responsibility,Inspection_Reason
0,2022-03-03,14T90P-G.AR3CL,202QCCV560955,OK,NaT,NaN,NaN,NaN,NaN,Supplier,품질안정화를 위한 전수검사
1,2022-03-03,14T90P-G.AR3CL,202QCCV561003,OK,NaT,NaN,NaN,NaN,NaN,Supplier,품질안정화를 위한 전수검사
2,2022-03-03,14T90P-G.AR3CL,202QCAS560968,OK,NaT,NaN,NaN,NaN,NaN,Supplier,품질안정화를 위한 전수검사
3,2022-03-03,14T90P-G.AR3CL,202QCFT561019,OK,NaT,NaN,NaN,NaN,NaN,Supplier,품질안정화를 위한 전수검사
4,2022-03-03,14T90P-G.AR3CL,202QCNL560967,OK,NaT,NaN,NaN,NaN,NaN,Supplier,품질안정화를 위한 전수검사
...,...,...,...,...,...,...,...,...,...,...,...
1854,2022-04-13,14TD90P-G.AXFBK,203QCKJ581469,OK,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1855,2022-04-13,14TD90P-G.AXFBK,203QCKJ581493,OK,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1856,2022-04-13,14TD90P-G.AXFBK,203QCEA581756,OK,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1857,2022-04-13,14TD90P-G.AXFBK,203QCFT581475,OK,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [157]:
df1.to_excel('trans.xlsx', index=False) # 엑셀로 만들어 번역 재검증

In [159]:
df1 = pd.read_excel(get_filename()) # 재검증한 결과를 다시 df로 만듬

In [32]:
df.columns

Index(['Inspection Date', 'Mapping Model.Suffix', 'S/N', 'Judge', 'MFG DATE',
       'Defect Reason_Korean', 'Defect Reason_English', 'Repair Report',
       'RMA Number', 'Responsibility', 'Inspection_Reason'],
      dtype='object')

In [59]:
df1

,Inspection Date,Mapping Model.Suffix,MFG DATE,S/N,Judge,Defect Reason_Korean,Responsibility,Inspection_Reason,Defect Reason_English
0,2022-08-23,14T90Q-G.AR5PK,2022-08-09,208QCGW560431,NG,C-Part Rubber 누락,Supplier,품질안정화를 위한 전수 검사,Missing C-Part Rubber
1,2022-08-23,14T90Q-G.AR5PK,2022-08-09,208QCAS560536,NG,Touch Pad 좌측 누름 시 상하 동시 눌림,Supplier,품질안정화를 위한 전수 검사,Upper and lower part simultaneously pressed wh...


In [4]:
df1 = pd.read_excel(get_filename(), sheet_name='NG')
df1

,Inspection Date,Mapping Model.Suffix,S/N,Judge,Defect Reason_Korean,Defect Reason_English,Responsibility,Inspection_Reason
0,2023-03-30,15UG50Q-G.AR56KN,303QCQX569420,NG,Box damage,Box damage,LG,Box Wet damage Issue
1,2023-03-30,15UG50Q-G.AR56KN,303QCQX569228,NG,Box damage,Box damage,LG,Box Wet damage Issue
2,2023-03-30,15UG50Q-G.AR56KN,303QCRN569534,NG,Box damage,Box damage,LG,Box Wet damage Issue
3,2023-03-30,15UG50Q-G.AR56KN,303QCSF569398,NG,Box damage,Box damage,LG,Box Wet damage Issue
4,2023-03-30,15UG50Q-G.AR56KN,303QCTB569233,NG,Box damage,Box damage,LG,Box Wet damage Issue
...,...,...,...,...,...,...,...,...
1058,2023-03-30,15UG50Q-G.AR56KN,303QCYQ569346,OK,NaN,NaN,LG,Box Wet damage Issue
1059,2023-03-30,15UG50Q-G.AR56KN,303QCYQ569370,OK,NaN,NaN,LG,Box Wet damage Issue
1060,2023-03-30,15UG50Q-G.AR56KN,303QCYQ569418,OK,NaN,NaN,LG,Box Wet damage Issue
1061,2023-03-30,15UG50Q-G.AR56KN,303QCXM569602,OK,NaN,NaN,LG,Box Wet damage Issue


## 3. 불량탭에서 가져온 데이타를 기존 DB에 추가함

In [6]:
with open('D:/Data/PC Inspection DB.bin', 'rb') as f:
    df = pickle.load(f)

In [7]:
df.shape

(108993, 12)

In [8]:
df1.shape

(1063, 8)

In [9]:
sample = pd.DataFrame() # DB에 update되는 신규 데이타만 취합하기 위해 만든 빈 dataframe
for row in tqdm(df1.iterrows()):
    cond = (df['Inspection Date'] == row[1]['Inspection Date']) & (df['S/N'] == row[1]['S/N']) & (df['Judge'] == row[1]['Judge']) & (df['Mapping Model.Suffix'] == row[1]['Mapping Model.Suffix'])
    if df[cond].shape[0] == 0:
        df = df.append(row[1])
        sample = sample.append(row[1])
df.reset_index(drop=True, inplace=True)

1063it [01:33, 11.39it/s]


In [10]:
sample.shape

(1063, 8)

In [11]:
df.shape

(110056, 12)

In [12]:
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])

In [13]:
df

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
0,2022-03-03,14T90P-G.AR3CL,202QCCV560955,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
1,2022-03-03,14T90P-G.AR3CL,202QCCV561003,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
2,2022-03-03,14T90P-G.AR3CL,202QCAS560968,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
3,2022-03-03,14T90P-G.AR3CL,202QCFT561019,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
4,2022-03-03,14T90P-G.AR3CL,202QCNL560967,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
110051,2023-03-30,15UG50Q-G.AR56KN,303QCYQ569346,OK,NaT,NaN,NaN,NaN,NaN,LG,Box Wet damage Issue,NaN
110052,2023-03-30,15UG50Q-G.AR56KN,303QCYQ569370,OK,NaT,NaN,NaN,NaN,NaN,LG,Box Wet damage Issue,NaN
110053,2023-03-30,15UG50Q-G.AR56KN,303QCYQ569418,OK,NaT,NaN,NaN,NaN,NaN,LG,Box Wet damage Issue,NaN
110054,2023-03-30,15UG50Q-G.AR56KN,303QCXM569602,OK,NaT,NaN,NaN,NaN,NaN,LG,Box Wet damage Issue,NaN


In [14]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

In [15]:
with open('D:/Data/DB backup/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

## 4. DOA 수리후 환입되는 정보를 DB에 추가

### 1) 업체별 DOA Return 현황 대비 입고 및 검사 수량 현황

In [5]:
with open('D:Data/PC ODM DOA return DB.bin', 'rb') as f:
    df1 = pickle.load(f)

In [13]:
with open('D:Data/PC ODM DOA return DB.bin', 'rb') as f:
    df1 = pickle.load(f)

df1 = df1.fillna('TBA').pivot_table('S/N', index=['Vendor', 'DOA Number', 'OBD'], aggfunc='count').reset_index()
df1 = df1.rename(columns={'S/N':'DOA QTY'})

with open('D:Data/SD_raw_data.bin', 'rb') as f:
    df2 = pickle.load(f)

df2 = df2[df2['Import Biz Type']=='NCV'].pivot_table('Shipment Quantity', index=['DOA Number'], aggfunc=sum).reset_index()
df2= df2.rename(columns={'Inspection':'DOA Number', 'Shipment Quantity':'Return QTY'})
df2

df3 = df1.merge(df2, how='left')
df3 = df3.fillna(0)
df3.sort_values(['Vendor', 'OBD'])

with open('D:/Data/PC Inspection DB.bin', 'rb') as f: # 전수검사 DB를 먼저 가져옴
    df4 = pickle.load(f)
df4 = df4[df4['DOA Number'].notnull()]

df4 = df4.pivot_table('S/N', index='DOA Number', aggfunc='count').reset_index()
df4 = df4.rename(columns={'S/N':'Inspection QTY'})

df5 = df3.merge(df4, how='left')
df5 = df5.fillna(0)
df5['Balance QTY'] = df5['DOA QTY'] - df5['Inspection QTY']
df5 = df5.convert_dtypes()
df5

,Vendor,DOA Number,OBD,DOA QTY,Return QTY,Inspection QTY,Balance QTY
0,Pegatron,LG20230410,TBA,14,0,0,14
1,Pegatron,LG211006,2021-10-12 00:00:00,35,35,35,0
2,Pegatron,LG211102,2021-11-09 00:00:00,26,26,26,0
3,Pegatron,LG211201,2021-12-09 00:00:00,63,63,63,0
4,Pegatron,LG220304,2022-03-17 00:00:00,51,51,51,0
5,Pegatron,LG220510,2022-05-25 00:00:00,27,27,27,0
6,Pegatron,LG220721,2022-08-01 00:00:00,39,39,39,0
7,Pegatron,LG220818,2022-09-01 00:00:00,562,562,552,10
8,Pegatron,LG221017,2022-10-23 00:00:00,42,42,42,0
9,Pegatron,LG221108,2022-11-13 00:00:00,45,45,45,0


### 2) 특정 DOA Number에 대한 모델별 입고/검사 수량 현황

In [4]:
doa_num = ['N5V22A08H-391L', 'N5V22C08H-391L']

In [5]:
with open('D:Data/PC ODM DOA return DB.bin', 'rb') as f:
    df1 = pickle.load(f)

df1 = df1[df1['DOA Number'].isin(doa_num)]
df1 = df1.pivot_table('S/N', index=['Mapping Model.Suffix'], aggfunc='count')
df1 = df1.rename(columns={'S/N':'DOA QTY'})

with open('D:Data/SD_raw_data.bin', 'rb') as f:
    df2 = pickle.load(f)

c1 = (df2['Import Biz Type']=='NCV')
c2 = (df2['DOA Number'].isin(doa_num))
if df2[c1 & c2].shape[0] != 0:
    df2 = df2[c1 & c2].pivot_table('Shipment Quantity', index='Item No', aggfunc=sum)
    df2 = df2.rename(columns={'Shipment Quantity':'Return QTY'})
else:
    df2 = pd.DataFrame(0, index=df1.index, columns=['Return QTY'])
    
with open('D:/Data/PC Inspection DB.bin', 'rb') as f: # 전수검사 DB를 먼저 가져옴
    df3 = pickle.load(f)

if df3[df3['DOA Number'].isin(doa_num)].shape[0] != 0:
    df3 = df3[df3['DOA Number'].isin(doa_num)]
    df3 = df3.pivot_table('S/N', index='Mapping Model.Suffix', aggfunc='count')
    df3 = df3.rename(columns={'S/N':'Inspected QTY'})
else:
    df3 = pd.DataFrame(0, index=df1.index, columns=['Inspected QTY'])

df3 = pd.concat([df1, df2, df3], axis=1)
df3 = df3.convert_dtypes()
df3 = df3.fillna(0)
df3['Balance QTY'] = df3['DOA QTY'] - df3['Inspected QTY']
df3.loc['Sum', :] = df3.sum()
df3

,DOA QTY,Return QTY,Inspected QTY,Balance QTY
11TC50Q-E.AC10ML,6,6,6,0
11TG50Q-E.AC10KN,21,21,21,0
14T90P-G.AA7DK,1,1,1,0
14T90P-G.AR5GK,1,1,1,0
14T90Q-G.AA79K,3,3,3,0
14T90Q-G.AR5GK,2,2,2,0
14T90Q-G.AR5PK,4,4,4,0
14TD90P-G.AX5GK,5,5,5,0
14TD90P-G.AX7GK,1,1,1,0
14TD90Q-G.AX5PK,7,7,7,0


### 3) DOA Number 중 아직 전수검사 하지 않은 S/N 리스트

In [6]:
def get_balance_doa_unit(doa_num):
    with open('D:/Data/PC ODM DOA return DB.bin', 'rb') as f:
        df1 = pickle.load(f)
    df1 = df1[df1['DOA Number'] == doa_num]
    
    with open('D:/Data/PC Inspection DB.bin', 'rb') as f:
        df2 = pickle.load(f)
    df2 = df2[df2['DOA Number'] == doa_num]
    df3 = df1[~df1['S/N'].isin(df2['S/N'])]
    return df3.reset_index(drop=True)

df = pd.DataFrame()
for lot in doa_num:
    df = pd.concat([df, get_balance_doa_unit(lot)])
df

,Vendor,Series,Mapping Model.Suffix,Price,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),FOC Number,Grouping Number,DOA Number,HBL,OBD,Remark
0,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCAS562552,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN
1,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCVU564071,T/Pad Button_L Noise,T/Pad Button_L Noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN
2,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCWC567233,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN
3,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCYQ558842,Set 내부 Noise,Set internal noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN
4,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCZP558413,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN
5,Quanta,15U50P,15U50P-K.AP5HL,554.1,202QCBD559365,Space Bar Noise,Space Bar Noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN
0,Quanta,15U50P,15UG50P-G.AP70KN,622.97,201QCWC557073,F11 초기화 안됨,F11 not initialized,F1000204203,CPCZ2212090001,N5V22C08H-391L,SEL-0055 2553,2022-12-14,NaN


In [10]:
df.pivot_table('S/N', index=['Mapping Model.Suffix', 'Price'], aggfunc='count')

,,S/N
Mapping Model.Suffix,Price,
15U50P-G.AP5VLF,507.51,5
15U50P-K.AP5HL,554.10,1
15UG50P-G.AP70KN,622.97,1


#### 실제 Quanta로부터 받은 DOA 발송 내역 불러옴

In [7]:
df2 = pd.read_excel(get_filename())
df2 = df2.rename(columns={'Quanta Action':'Repair Report'})
df2

,Vendor,Series,Mapping Model.Suffix,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),Remark,Unnamed: 7,Unnamed: 8,Repair Report
0,Quanta,15U50P,15U50P-G.AP5VLF,112QCAS562552,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),NaN,NaN,NaN,"Replace LCD, update DPK"
1,Quanta,15U50P,15U50P-G.AP5VLF,112QCVU564071,T/Pad Button_L Noise,T/Pad Button_L Noise,NaN,NaN,NaN,"Replace C cover, update DPK"
2,Quanta,15U50P,15U50P-G.AP5VLF,112QCWC567233,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),NaN,NaN,NaN,"Replace LCD, update DPK"
3,Quanta,15U50P,15U50P-G.AP5VLF,112QCYQ558842,Set 내부 Noise,Set internal noise,NaN,NaN,NaN,"Repair MB, update DPK"
4,Quanta,15U50P,15U50P-G.AP5VLF,112QCZP558413,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),NaN,NaN,NaN,"Replace LCD, update DPK"
5,Quanta,15U50P,15U50P-K.AP5HL,202QCBD559365,Space Bar Noise,Space Bar Noise,NaN,NaN,NaN,"Replace C cover, update DPK"
6,Quanta,NaN,15UG50P-G.AP70KN,201QCWC557073,F11 초기화 안됨,F11 not initialized,NaN,NaN,NaN,"Replace C cover, update DPK"


In [8]:
df3 = df[df['S/N'].isin(df2['S/N'])]

In [9]:
df3 = pd.merge(df3, df2[['S/N', 'Repair Report']])
df3

,Vendor,Series,Mapping Model.Suffix,Price,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),FOC Number,Grouping Number,DOA Number,HBL,OBD,Remark,Repair Report
0,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCAS562552,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace LCD, update DPK"
1,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCVU564071,T/Pad Button_L Noise,T/Pad Button_L Noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace C cover, update DPK"
2,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCWC567233,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace LCD, update DPK"
3,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCYQ558842,Set 내부 Noise,Set internal noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Repair MB, update DPK"
4,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCZP558413,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace LCD, update DPK"
5,Quanta,15U50P,15U50P-K.AP5HL,554.1,202QCBD559365,Space Bar Noise,Space Bar Noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace C cover, update DPK"
6,Quanta,15U50P,15UG50P-G.AP70KN,622.97,201QCWC557073,F11 초기화 안됨,F11 not initialized,F1000204203,CPCZ2212090001,N5V22C08H-391L,SEL-0055 2553,2022-12-14,NaN,"Replace C cover, update DPK"


In [10]:
inv = df3.groupby(['Mapping Model.Suffix', 'Price'])['S/N'].count()
inv.name = 'QTY'
inv = inv.reset_index()
inv['Amt'] = inv['Price'] * inv['QTY']
print('Amount :', inv['Amt'].sum(), 'Total Qty :', inv['QTY'].sum())
inv

Amount : 3714.62 Total Qty : 7


,Mapping Model.Suffix,Price,QTY,Amt
0,15U50P-G.AP5VLF,507.51,5,2537.55
1,15U50P-K.AP5HL,554.10,1,554.10
2,15UG50P-G.AP70KN,622.97,1,622.97


In [17]:
with pd.ExcelWriter('D:/Python result/Quanta DOA return list.xlsx') as writer:
    df3.to_excel(writer, sheet_name='RAW DATA', index=False)
    inv.to_excel(writer, sheet_name='Inv_data', index=False)

In [8]:
df4 = pd.read_excel(get_filename(), sheet_name=1)
df4 = df4[['S/N', 'Repair Report']]
df4

,S/N,Repair Report
0,206QCAS574800,Repair MB
1,206QCEA574724,Replace B cover
2,206QCGW574767,Replace C cover
3,206QCGW574815,Replace D cover
4,206QCHE566289,Repair MB
5,206QCLH574672,Replace B cover
6,206QCLH574720,Repair MB
7,206QCPY574616,Replace B cover
8,206QCRN574638,Replace LCD
9,206QCSF574694,Replace B cover


In [44]:
# Pegatron의 경우
df4 = pd.read_excel(get_filename(), sheet_name=0)
df4.rename(columns={'SSN':'S/N', 'Repair method':'Repair Report'}, inplace=True)
df4 = df4[['S/N', 'Repair Report']]
df4

,S/N,Repair Report
0,205PGAG720328,Change bezel
1,205PGAG720376,Change bezel
2,205PGCD720315,Change bezel
3,205PGDX720322,reassemble the power key
4,205PGEF720324,Change bezel
5,205PGEF720396,Change bezel
6,205PGFJ720331,Change bezel
7,205PGFJ720427,Change bezel
8,205PGJK720308,Change bezel
9,205PGKW720397,Change bezel; repair MB acetate tape


In [11]:
# Quanta의 경우
df4 = pd.read_excel(get_filename(), sheet_name=0, skiprows=0)
df4 = df4[df4['S/N'].notnull()]
df4.rename(columns={'Quanta Action':'Repair Report'}, inplace=True)
df4

,Vendor,Series,Mapping Model.Suffix,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),Remark,Unnamed: 7,Unnamed: 8,Repair Report
0,Quanta,15U50P,15U50P-G.AP5VLF,112QCAS562552,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),NaN,NaN,NaN,"Replace LCD, update DPK"
1,Quanta,15U50P,15U50P-G.AP5VLF,112QCVU564071,T/Pad Button_L Noise,T/Pad Button_L Noise,NaN,NaN,NaN,"Replace C cover, update DPK"
2,Quanta,15U50P,15U50P-G.AP5VLF,112QCWC567233,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),NaN,NaN,NaN,"Replace LCD, update DPK"
3,Quanta,15U50P,15U50P-G.AP5VLF,112QCYQ558842,Set 내부 Noise,Set internal noise,NaN,NaN,NaN,"Repair MB, update DPK"
4,Quanta,15U50P,15U50P-G.AP5VLF,112QCZP558413,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),NaN,NaN,NaN,"Replace LCD, update DPK"
5,Quanta,15U50P,15U50P-K.AP5HL,202QCBD559365,Space Bar Noise,Space Bar Noise,NaN,NaN,NaN,"Replace C cover, update DPK"
6,Quanta,NaN,15UG50P-G.AP70KN,201QCWC557073,F11 초기화 안됨,F11 not initialized,NaN,NaN,NaN,"Replace C cover, update DPK"


In [12]:
# 아직 돌려보내지 않은 DOA set 중 이번에 들어온 S/N 리스트에 해당하는 수량이 맞는지 확인
df3 = df3[df3['S/N'].isin(df4['S/N'])]
df3

,Vendor,Series,Mapping Model.Suffix,Price,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),FOC Number,Grouping Number,DOA Number,HBL,OBD,Remark,Repair Report
0,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCAS562552,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace LCD, update DPK"
1,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCVU564071,T/Pad Button_L Noise,T/Pad Button_L Noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace C cover, update DPK"
2,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCWC567233,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace LCD, update DPK"
3,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCYQ558842,Set 내부 Noise,Set internal noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Repair MB, update DPK"
4,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCZP558413,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace LCD, update DPK"
5,Quanta,15U50P,15U50P-K.AP5HL,554.1,202QCBD559365,Space Bar Noise,Space Bar Noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace C cover, update DPK"
6,Quanta,15U50P,15UG50P-G.AP70KN,622.97,201QCWC557073,F11 초기화 안됨,F11 not initialized,F1000204203,CPCZ2212090001,N5V22C08H-391L,SEL-0055 2553,2022-12-14,NaN,"Replace C cover, update DPK"


In [11]:
inv = df3.groupby(['Mapping Model.Suffix', 'Price'])['S/N'].count().reset_index()
inv.rename(columns={'S/N':'QTY'}, inplace=True)
inv['Amt'] = inv['Price'] * inv['QTY']
inv['QTY'].sum(), inv['Amt'].sum()

(45, 18065.65)

In [34]:
inv.to_clipboard()

In [10]:
df4 = df4[['S/N', 'Repair Report']]

In [10]:
df3 = df3.merge(df4, how='right')

In [11]:
df3

,Vendor,Series,Mapping Model.Suffix,Price,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),FOC Number,Grouping Number,DOA Number,HBL,OBD,Remark,Repair Report
0,Quanta,11T50Q,11TG50Q-E.AC10KN,300.17,206QCAS574800,간헐적 인터넷 끊김,Intermittent Internet Loss,F1000203479,CPCZ2211160006,N5V22B10H-391L,550115928366,2022-11-20,NaN,Repair MB
1,Quanta,11T50Q,11TG50Q-E.AC10KN,300.17,206QCEA574724,LCD Open/Close Noise,LCD Open/Close Noise,F1000203479,CPCZ2211160006,N5V22B10H-391L,550115928366,2022-11-20,NaN,Replace B cover
2,Quanta,11T50Q,11TG50Q-E.AC10KN,300.17,206QCGW574767,C-Part Scratch,C-Part Scratch,F1000203479,CPCZ2211160006,N5V22B10H-391L,550115928366,2022-11-20,NaN,Replace C cover
3,Quanta,11T50Q,11TG50Q-E.AC10KN,300.17,206QCGW574815,D-Part Scratch,D-Part Scratch,F1000203479,CPCZ2211160006,N5V22B10H-391L,550115928366,2022-11-20,NaN,Replace D cover
4,Quanta,11T50Q,11TG50Q-E.AC10KN,300.17,206QCHE566289,제품 내부 소음,product internal noise,F1000203479,CPCZ2211160006,N5V22B10H-391L,550115928366,2022-11-20,NaN,Repair MB
5,Quanta,11T50Q,11TG50Q-E.AC10KN,300.17,206QCLH574672,LCD Open/Close Noise,LCD Open/Close Noise,F1000203479,CPCZ2211160006,N5V22B10H-391L,550115928366,2022-11-20,NaN,Replace B cover
6,Quanta,11T50Q,11TG50Q-E.AC10KN,300.17,206QCLH574720,KBD Alt Key 복원 느림,KBD Alt Key restore slow,F1000203479,CPCZ2211160006,N5V22B10H-391L,550115928366,2022-11-20,NaN,Repair MB
7,Quanta,11T50Q,11TG50Q-E.AC10KN,300.17,206QCPY574616,LCD Open/Close Noise,LCD Open/Close Noise,F1000203479,CPCZ2211160006,N5V22B10H-391L,550115928366,2022-11-20,NaN,Replace B cover
8,Quanta,11T50Q,11TG50Q-E.AC10KN,300.17,206QCRN574638,화면 회전 안됨,Screen not rotating,F1000203479,CPCZ2211160006,N5V22B10H-391L,550115928366,2022-11-20,NaN,Replace LCD
9,Quanta,11T50Q,11TG50Q-E.AC10KN,300.17,206QCSF574694,LCD Open/Close Noise,LCD Open/Close Noise,F1000203479,CPCZ2211160006,N5V22B10H-391L,550115928366,2022-11-20,NaN,Replace B cover


In [26]:
# 수리 후 환입 리스트를 파일로 출력
df3.drop(columns=['FOC Number', 'Grouping Number', 'HBL', 'OBD', 'Remark']).to_excel('D:/Python result/221220_Quanta DOA 수리 후 환입내역_42대.xlsx')

### 4) 수리내역에 대한 정확한 Invoice 수량/금액 확인

In [14]:
# 수리내역에 대한 정확한 invoice 수량/금액 확인
df4 = df3.pivot_table('S/N', aggfunc='count', index=['Mapping Model.Suffix', 'Price']).reset_index()
df4 = df4.rename(columns={'S/N':'QTY'})
df4['Amount'] = df4['Price'] * df4['QTY']
df4

,Mapping Model.Suffix,Price,QTY,Amount
0,11TC50Q-E.AC10ML,295.17,1,295.17
1,11TG50Q-E.AC10KN,300.17,21,6303.57
2,14T90Q-G.AR5PK,738.29,1,738.29
3,15UD50Q-G.AX50K,484.86,5,2424.30
4,16T90Q-G.AR5BK,786.75,2,1573.50
5,16TG90P-G.AR5DKN,781.69,7,5471.83


In [15]:
df4['Amount'].sum()

16806.660000000003

In [6]:
df3 = pd.read_excel(get_filename())

### 4) DOA 환입품 전수검사 내역의 S/N과 일치하는 지 확인

In [13]:
# DOA 전수검사 결과 파일 불러옴
df5 = pd.read_excel(get_filename(), skiprows=2, sheet_name=1)
df5 = df5.loc[:, '모델명':'불량 내용']
df5

,모델명,S/N,생산지,차용일,불량 증상,수리 내용,판정,불량 내용
0,15U50P-G.AP5VLF,112QCAS562552,Quanta,2023-03-07,LCD B/M 불량(좌/우 편차 1.5mm↑),"Replace LCD, update DPK",OK,NaN
1,15U50P-G.AP5VLF,112QCVU564071,Quanta,2023-03-07,T/Pad Button_L Noise,"Replace C cover, update DPK",OK,NaN
2,15U50P-G.AP5VLF,112QCWC567233,Quanta,2023-03-07,LCD B/M 불량(좌/우 편차 1.5mm↑),"Replace LCD, update DPK",NG,A-Part Dent
3,15U50P-G.AP5VLF,112QCYQ558842,Quanta,2023-03-07,Set 내부 Noise,"Repair MB, update DPK",OK,NaN
4,15U50P-G.AP5VLF,112QCZP558413,Quanta,2023-03-07,LCD B/M 불량(좌/우 편차 1.5mm↑),"Replace LCD, update DPK",OK,NaN
5,15U50P-K.AP5HL,202QCBD559365,Quanta,2023-03-07,Space Bar Noise,"Replace C cover, update DPK",OK,NaN
6,15UG50P-G.AP70KN,201QCWC557073,Quanta,2023-03-07,F11 초기화 안됨,"Replace C cover, update DPK",OK,NaN


In [14]:
# 불러온 파일의 S/N 리스트와 업체가 보낸 S/N 리스트와 일치하는지 확인
set(df5['S/N']) - set(df3['S/N'])

set()

### 6) 전수검사 DB에 들어갈 데이타프레임 작성

In [69]:
df3 = pd.merge(df3, df5[['S/N', '수리 내용']])

In [70]:
df3 = df3.rename(columns={'수리 내용':'Repair Report'})

In [17]:
df5

,모델명,S/N,생산지,차용일,불량 증상,수리 내용,판정,불량 내용
0,15U50P-G.AP5VLF,112QCAS562552,Quanta,2023-03-07,LCD B/M 불량(좌/우 편차 1.5mm↑),"Replace LCD, update DPK",OK,NaN
1,15U50P-G.AP5VLF,112QCVU564071,Quanta,2023-03-07,T/Pad Button_L Noise,"Replace C cover, update DPK",OK,NaN
2,15U50P-G.AP5VLF,112QCWC567233,Quanta,2023-03-07,LCD B/M 불량(좌/우 편차 1.5mm↑),"Replace LCD, update DPK",NG,A-Part Dent
3,15U50P-G.AP5VLF,112QCYQ558842,Quanta,2023-03-07,Set 내부 Noise,"Repair MB, update DPK",OK,NaN
4,15U50P-G.AP5VLF,112QCZP558413,Quanta,2023-03-07,LCD B/M 불량(좌/우 편차 1.5mm↑),"Replace LCD, update DPK",OK,NaN
5,15U50P-K.AP5HL,202QCBD559365,Quanta,2023-03-07,Space Bar Noise,"Replace C cover, update DPK",OK,NaN
6,15UG50P-G.AP70KN,201QCWC557073,Quanta,2023-03-07,F11 초기화 안됨,"Replace C cover, update DPK",OK,NaN


In [71]:
df3 = df3[['Mapping Model.Suffix', 'S/N', 'DOA Number', 'Repair Report']]

In [72]:
df3

,Mapping Model.Suffix,S/N,DOA Number,Repair Report,Repair Report
0,15U40Q-G.AR30K,109QCKJ560941,N5V22A08H-391L,Replace B cover,Replace B cover
1,15U40Q-G.AR30K,111QCUK570270,N5V22A08H-391L,Replace Speaker,Replace Speaker
2,15U50P-G.AR3JK,110QCCV568619,N5V22A08H-391L,Replace T/Pad,Replace T/Pad
3,15U50P-G.AR50ML,201QCRN577358,N5V22A08H-391L,Replace Speaker,Replace Speaker
4,11TG50Q-E.AC10KN,206QCAS574704,N5V22C08H-391L,Replace T/Pad,Replace T/Pad
...,...,...,...,...,...
76,16T90Q-G.AAFBK,211QCJZ561380,N5V22C08H-391L,Replace LCD,Replace LCD
77,16TD90Q-G.AX70K,204QCVU564399,N5V22C08H-391L,Replace D cover,Replace D cover
78,16TD90Q-G.AX70K,209QCEA558332,N5V22C08H-391L,Replace B cover,Replace B cover
79,16TD90Q-G.AX70K,211QCUK561542,N5V22C08H-391L,Replace hinge,Replace hinge


In [19]:
df5 = df5[['S/N', '판정', '불량 내용']]
df5 = df5.rename(columns={'판정':'Judge', '불량 내용':'Defect Reason_Korean'})

In [20]:
df5

,S/N,Judge,Defect Reason_Korean
0,112QCAS562552,OK,NaN
1,112QCVU564071,OK,NaN
2,112QCWC567233,NG,A-Part Dent
3,112QCYQ558842,OK,NaN
4,112QCZP558413,OK,NaN
5,202QCBD559365,OK,NaN
6,201QCWC557073,OK,NaN


In [22]:
df3 = df3.merge(df5)
df3

,Vendor,Series,Mapping Model.Suffix,Price,S/N,Defective_Symptom(KOR),Defective_Symptom(ENG),FOC Number,Grouping Number,DOA Number,HBL,OBD,Remark,Repair Report,Judge,Defect Reason_Korean
0,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCAS562552,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace LCD, update DPK",OK,NaN
1,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCVU564071,T/Pad Button_L Noise,T/Pad Button_L Noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace C cover, update DPK",OK,NaN
2,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCWC567233,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace LCD, update DPK",NG,A-Part Dent
3,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCYQ558842,Set 내부 Noise,Set internal noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Repair MB, update DPK",OK,NaN
4,Quanta,15U50P,15U50P-G.AP5VLF,507.51,112QCZP558413,LCD B/M 불량(좌/우 편차 1.5mm↑),Bad LCD B/M (left/right deviation 1.5mm ↑),F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace LCD, update DPK",OK,NaN
5,Quanta,15U50P,15U50P-K.AP5HL,554.1,202QCBD559365,Space Bar Noise,Space Bar Noise,F1000202340,CPCZ2210170001,N5V22A08H-391L,SEL-0054 5657,2022-10-20,NaN,"Replace C cover, update DPK",OK,NaN
6,Quanta,15U50P,15UG50P-G.AP70KN,622.97,201QCWC557073,F11 초기화 안됨,F11 not initialized,F1000204203,CPCZ2212090001,N5V22C08H-391L,SEL-0055 2553,2022-12-14,NaN,"Replace C cover, update DPK",OK,NaN


In [26]:
df5.columns

Index(['Inspection Date', 'Mapping Model.Suffix', 'S/N', 'Judge', 'MFG DATE',
       'Defect Reason_Korean', 'Defect Reason_English', 'Repair Report',
       'DOA Number', 'Responsibility', 'Inspection_Reason', 'Defect Category'],
      dtype='object')

In [ ]:
df3.columns = 

In [23]:
with open('D:/Data/PC Inspection DB.bin', 'rb') as f:
    df1 = pickle.load(f)

In [24]:
df5 = pd.DataFrame(columns=df1.columns)
df5

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category


In [25]:
df5 = pd.concat([df5, df3])[df5.columns]
df5.head(1)

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
0,NaN,15U50P-G.AP5VLF,112QCAS562552,OK,NaN,NaN,NaN,"Replace LCD, update DPK",N5V22A08H-391L,NaN,NaN,NaN


In [28]:
df5

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
0,NaN,15U50P-G.AP5VLF,112QCAS562552,OK,NaN,NaN,NaN,"Replace LCD, update DPK",N5V22A08H-391L,NaN,NaN,NaN
1,NaN,15U50P-G.AP5VLF,112QCVU564071,OK,NaN,NaN,NaN,"Replace C cover, update DPK",N5V22A08H-391L,NaN,NaN,NaN
2,NaN,15U50P-G.AP5VLF,112QCWC567233,NG,NaN,A-Part Dent,NaN,"Replace LCD, update DPK",N5V22A08H-391L,NaN,NaN,NaN
3,NaN,15U50P-G.AP5VLF,112QCYQ558842,OK,NaN,NaN,NaN,"Repair MB, update DPK",N5V22A08H-391L,NaN,NaN,NaN
4,NaN,15U50P-G.AP5VLF,112QCZP558413,OK,NaN,NaN,NaN,"Replace LCD, update DPK",N5V22A08H-391L,NaN,NaN,NaN
5,NaN,15U50P-K.AP5HL,202QCBD559365,OK,NaN,NaN,NaN,"Replace C cover, update DPK",N5V22A08H-391L,NaN,NaN,NaN
6,NaN,15UG50P-G.AP70KN,201QCWC557073,OK,NaN,NaN,NaN,"Replace C cover, update DPK",N5V22C08H-391L,NaN,NaN,NaN


In [56]:
print(f"DOA 검사 불량율 : {df5.groupby('Judge')['S/N'].count()['NG'] / (df5.groupby('Judge')['S/N'].count()['OK'] + df5.groupby('Judge')['S/N'].count()['NG']) * 100:.2f} %")

DOA 검사 불량율 : 20.99 %


In [29]:
# 변수 설정
ins_date = datetime.datetime(2023,3,13) # 검사일 설정
rsps = 'LG'
Insp_reason = 'DOA units inspection'

In [30]:
df5['Inspection Date'] = ins_date
df5['Responsibility'] = rsps
df5['Inspection_Reason'] = Insp_reason
df5 # 전수검사 DB에 업데이트할 금번 DOA 전수검사 결과 리스트

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
0,2023-03-13,15U50P-G.AP5VLF,112QCAS562552,OK,NaN,NaN,NaN,"Replace LCD, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN
1,2023-03-13,15U50P-G.AP5VLF,112QCVU564071,OK,NaN,NaN,NaN,"Replace C cover, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN
2,2023-03-13,15U50P-G.AP5VLF,112QCWC567233,NG,NaN,A-Part Dent,NaN,"Replace LCD, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN
3,2023-03-13,15U50P-G.AP5VLF,112QCYQ558842,OK,NaN,NaN,NaN,"Repair MB, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN
4,2023-03-13,15U50P-G.AP5VLF,112QCZP558413,OK,NaN,NaN,NaN,"Replace LCD, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN
5,2023-03-13,15U50P-K.AP5HL,202QCBD559365,OK,NaN,NaN,NaN,"Replace C cover, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN
6,2023-03-13,15UG50P-G.AP70KN,201QCWC557073,OK,NaN,NaN,NaN,"Replace C cover, update DPK",N5V22C08H-391L,LG,DOA units inspection,NaN


In [31]:
df5['Defect Reason_Korean'] = df5['Defect Reason_Korean'].str.replace('\n', ',') # 불량 증상 문자열 중 '/n' 이 있으면 ',' 로 변경함

In [32]:
df5.loc[df5['Defect Reason_Korean'].notnull(), ['Defect Reason_Korean', 'Defect Reason_English']] # 불량 증상만 나온 열을 확인

,Defect Reason_Korean,Defect Reason_English
2,A-Part Dent,NaN


In [33]:
df5.loc[df5['Defect Reason_Korean'].notnull(), 'Defect Reason_English'] = df5[df5['Defect Reason_Korean'].notnull()]['Defect Reason_Korean'].apply(get_google_translate, args=['en']) # Google 번역
df5[df5['Defect Reason_Korean'].notnull()]

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
2,2023-03-13,15U50P-G.AP5VLF,112QCWC567233,NG,NaN,A-Part Dent,A-Part Dent,"Replace LCD, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN


In [34]:
df5.loc[df5['Defect Reason_Korean'].notnull(), ('Defect Reason_Korean', 'Defect Reason_English')]

,Defect Reason_Korean,Defect Reason_English
2,A-Part Dent,A-Part Dent


In [81]:
df5.loc[df5['Defect Reason_Korean'].str.contains('작음', case=False, na=False), ('Defect Reason_Korean', 'Defect Reason_English')]

,Defect Reason_Korean,Defect Reason_English
3,Speaker-R Sound 작음,Speaker-R sound weak


In [80]:
df5.loc[df5['Defect Reason_Korean'].str.contains('작음', case=False, na=False), 'Defect Reason_English'] = 'Speaker-R sound weak'

In [82]:
df5[df5['Defect Reason_Korean'].notnull()]

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
1,2023-01-28,15U40Q-G.AR30K,111QCUK570270,NG,NaN,Speaker-R Noise,Speaker-R Noise,Replace Speaker,N5V22A08H-391L,LG,DOA units inspection,NaN
3,2023-01-28,15U50P-G.AR50ML,201QCRN577358,NG,NaN,Speaker-R Sound 작음,Speaker-R sound weak,Replace Speaker,N5V22A08H-391L,LG,DOA units inspection,NaN
4,2023-01-28,11TG50Q-E.AC10KN,206QCAS574704,NG,NaN,T/Pad Button-L 둔감,T/Pad Button-L insensitive,Replace T/Pad,N5V22C08H-391L,LG,DOA units inspection,NaN
6,2023-01-28,14T90Q-G.AA79K,210QCAS557192,NG,NaN,A/B-Part 상단 중앙 누름시 소음,Noise when pressing the center of the top of A/B-Part,Replace B cover,N5V22C08H-391L,LG,DOA units inspection,NaN
11,2023-01-28,15U40Q-G.ARFWK,202QCXM581826,NG,NaN,Speaker-L Noise,Speaker-L Noise,"Replace Speaker, Box",N5V22C08H-391L,LG,DOA units inspection,NaN
12,2023-01-28,15U40Q-G.ARTWK,207QCMR560187,NG,NaN,Mic Noise(녹음 후 재생시),Mic Noise (when playing back after recording),Repair MB,N5V22C08H-391L,LG,DOA units inspection,NaN
14,2023-01-28,15UD50Q-G.AX50K,209QCEA556916,NG,NaN,Box 불량,Box bad,Replace C cover,N5V22C08H-391L,LG,DOA units inspection,NaN
16,2023-01-28,15UD50Q-G.AX50K,210QCEA561508,NG,NaN,Box 불량,Box bad,Replace C cover,N5V22C08H-391L,LG,DOA units inspection,NaN
18,2023-01-28,16T90Q-G.AA56K,204QCEA562372,NG,NaN,B-Part 우측 하단부 조립 불량,Incorrect assembly of the lower right part of B-Part,Replace Hinge cap,N5V22C08H-391L,LG,DOA units inspection,NaN
34,2023-01-28,16T90Q-G.AA79K,210QCYQ562050,NG,NaN,B-Part 돌출,Extrude B-Part,Replace thicker back glue,N5V22C08H-391L,LG,DOA units inspection,NaN


In [35]:
with open('D:/Data/PC Inspection DB.bin', 'rb') as f: # 전수검사 DB를 먼저 가져옴
    df = pickle.load(f)

In [36]:
df.shape

(110056, 12)

In [37]:
df5.shape

(7, 12)

In [38]:
df = pd.concat([df, df5]).reset_index(drop=True)
df

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,DOA Number,Responsibility,Inspection_Reason,Defect Category
0,2022-03-03,14T90P-G.AR3CL,202QCCV560955,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
1,2022-03-03,14T90P-G.AR3CL,202QCCV561003,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
2,2022-03-03,14T90P-G.AR3CL,202QCAS560968,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
3,2022-03-03,14T90P-G.AR3CL,202QCFT561019,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
4,2022-03-03,14T90P-G.AR3CL,202QCNL560967,OK,NaT,NaN,NaN,NaN,NaN,LG,Full inspection for quality control,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
110058,2023-03-13,15U50P-G.AP5VLF,112QCWC567233,NG,NaT,A-Part Dent,A-Part Dent,"Replace LCD, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN
110059,2023-03-13,15U50P-G.AP5VLF,112QCYQ558842,OK,NaT,NaN,NaN,"Repair MB, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN
110060,2023-03-13,15U50P-G.AP5VLF,112QCZP558413,OK,NaT,NaN,NaN,"Replace LCD, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN
110061,2023-03-13,15U50P-K.AP5HL,202QCBD559365,OK,NaT,NaN,NaN,"Replace C cover, update DPK",N5V22A08H-391L,LG,DOA units inspection,NaN


In [39]:
df.shape

(110063, 12)

In [40]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

In [41]:
with open('D:/Data/DB backup/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

### SD에 RMA Number가 없으면 기입

In [109]:
rma_num = 'N5V21927H-391'
df1.loc['889110028321', 'Inspection'] = rma_num

In [111]:
df1.shape

(1781, 38)

In [112]:
with open('D:/Data/SD_raw_data.bin', 'wb') as f:
    pickle.dump(df1, f)

## 해당 DOA 넘버에 해당하는 S/N 리스트 추출

In [72]:
with open('D:Data/PC ODM DOA return DB.bin', 'rb') as f:
    df1 = pickle.load(f)
rma_num = 'N5V22316H-391L'
df1 = df1[df1['RMA Number'] == rma_num] # ODM 업체의 RMA 넘버에 해당하는 S/N 만 추출
df1.head(3)

,S/N,Vendor,Series,Mapping Model.Suffix,Price,Defective_Symptom(KOR),Defective_Symptom(ENG),FOC Number,Grouping Number,RMA Number,HBL,OBD,Remark
1017,201QCAS589256,Quanta,11T50Q,11TC50Q-E.AC10K,311.44,C-Part Scratch & Dent,C-Part Scratch & Dent,F1000195776,CPCZ2203240001,N5V22316H-391L,SEL-0053 2825,2022-04-14,NaN
1018,201QCAS589904,Quanta,11T50Q,11TC50Q-E.AC10K,311.44,T/P 우측 상단 들뜸\nAccessory Partition 누락,T/P upper right is lifted_x000D_\nAccessory Pa...,F1000195776,CPCZ2203240001,N5V22316H-391L,SEL-0053 2825,2022-04-14,NaN
1019,201QCAS594944,Quanta,11T50Q,11TC50Q-E.AC10K,311.44,A-Part 얼룩,A-Part Stain,F1000195776,CPCZ2203240001,N5V22316H-391L,SEL-0053 2825,2022-04-14,NaN


In [73]:
df1.shape

(261, 13)

In [299]:
# 업체(Pegatron, Quanta의 엑셀 수리 내역을 복사하여 데이타프레임으로 만듬
df2 = pd.read_clipboard()
df2.head(3)

,S/N,QTA PN,Vendor,Model,Price,FOC Number,Grouping Number,RMA Number,Repair Report,Cost($)
0,112QCNL583575,1NLH0000318,Quanta,14T90P-G.AA7BK,845.13,F1000195776,CPCZ2203240001,N5V22316H-391L,Replace D cover,3
1,201QCLH580008,1NLH0000318,Quanta,14T90P-G.AA7BK,845.13,F1000195776,CPCZ2203240001,N5V22316H-391L,Replace D cover,3
2,112QCBD592269,1NLH0000315,Quanta,14T90P-G.AA7DK,846.53,F1000195776,CPCZ2203240001,N5V22316H-391L,Replace LCD,12


In [300]:
# S/N 자리수 check 하여 이상한 것 있으면 공백 제거함
df2.insert(1, 'S/N count', df2['S/N'].str.len())
df2[df2['S/N count']!=13]['S/N'].tolist()

[]

In [301]:
df2['S/N'] = df2['S/N'].str.strip() # 이상한 것이 발견되면 공백 제거 실시
df2['S/N count'] = df2['S/N'].str.len() # 공백 제거 후, 다시 자리수 check
df2[df2['S/N count']!=13]['S/N'].tolist() # 결과 확인

[]

In [302]:
# DOA return 한 S/N에 해당하는 업체의 각 S/N별 수리 내역을 Merge하여 새로운 df로 만듬
df3 = df1[['Mapping Model.Suffix', 'S/N', 'Defective_Symptom(KOR)', 'Defective_Symptom(ENG)']].merge(df2[['S/N', 'Repair Report']], how='outer')

In [303]:
df3['RMA Number'] = rma_num

In [304]:
df3[df3['Repair Report'].notnull()].shape # 만들어진 df 내용 확인]

(54, 6)

In [305]:
df3 = df3[df3['Repair Report'].notnull()]

### 특정 DOA number의 return한 S/N 개수에서 지금까지 입고되어 전수검사 완료된 S/N 제외해봄

In [85]:
# DOA Return DB에서 특정 DOA number로 Return 한 S/N 개수 확인
with open('D:Data/PC ODM DOA return DB.bin', 'rb') as f:
    df1 = pickle.load(f)
df1 = df1[df1['RMA Number'] == rma_num]
df1.shape

(261, 13)

In [83]:
# 전수검사 DB에서 특정 DOA 로 들어온 전체 S/N 개수 확인
rma_num = 'N5V22316H-391L'

with open('D:/Data/PC Inspection DB.bin', 'rb') as f: # 전수검사 DB를 먼저 가져옴
    df = pickle.load(f)
df = df[df['RMA Number'] == rma_num]
df.shape

(119, 11)

In [88]:
print(f'아직 검사 안한 S/N 개수 : {df1.shape[0] - df.shape[0]}' )

아직 검사 안한 S/N 개수 : 142


In [308]:
blank_list = []
for col in df3.columns:
    if col in df.columns:
        blank_list.append(col)
blank_list

['Mapping Model.Suffix', 'S/N', 'Repair Report', 'RMA Number']

In [309]:
df = pd.concat([df, df3[blank_list]])

In [310]:
df = df.reset_index(drop=True) # 인덱스 재부여

In [311]:
df.shape

(83495, 11)

In [ ]:
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])

In [312]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)

### 2) 해당 수리 LOT에 입고검사 결과 붙이기

In [315]:
inspection_date = datetime.date(2022,6,18)

In [324]:
cond1 = df['RMA Number'] == rma_num

In [318]:
df[cond].shape

(119, 11)

In [319]:
# 입고 검사 결과 엑셀파일을 불러와서 불량율 맞는지 확인해보기
df_new = pd.read_clipboard()
df_new['판정'].value_counts(normalize=True)

OK    0.87037
NG    0.12963
Name: 판정, dtype: float64

In [320]:
df_new['S/N'].str.len().value_counts() # S/N 자리수가 13자리 맞는지 확인

13    54
Name: S/N, dtype: int64

In [321]:
df_new.columns

Index(['모델명', 'S/N', '생산지', '차용일', '불량 증상', '수리 내용', '판정', '불량 내용', '반납일'], dtype='object')

In [322]:
for sn, model, judge, defect in df_new[['S/N', '모델명', '판정', '불량 내용']].values:
    cond1 = (df['RMA Number'] == rma_num)
    cond2 = (df['S/N'] == sn)
    cond3 = (df['Mapping Model.Suffix'] == model)
    df.loc[cond1&cond2&cond3, ['Inspection Date', 'Judge', 'Defect Reason_Korean', 'Responsibility', 'Inspection_Reason']] = [
        inspection_date, judge, defect, 'LG', 'DOA units inspection']

In [323]:
cond4 = df['Defect Reason_Korean'].notnull()

In [325]:
df[cond1&cond4]

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,RMA Number,Responsibility,Inspection_Reason
83454,2022-06-18,14T90P-G.AAF6K,112QCZP588845,NG,NaT,LCD Open/Close Noise,NaN,Repair MB,N5V22316H-391L,LG,DOA units inspection
83456,2022-06-18,14T90P-G.AR5BK,201QCMR555915,NG,NaT,D-Part Dent,NaN,Replace C cover,N5V22316H-391L,LG,DOA units inspection
83460,2022-06-18,14T90P-G.AR5GK,112QCWC587345,NG,NaT,"D-Part 돌출, Box 불량",NaN,Replace T/P,N5V22316H-391L,LG,DOA units inspection
83461,2022-06-18,14T90P-G.AR5GK,112QCYQ583994,NG,NaT,D-Part 소음,NaN,Replace B cover,N5V22316H-391L,LG,DOA units inspection
83470,2022-06-18,14T90P-G.AR5GK,202QCRN561094,NG,NaT,B-Part Scratch/Dent,NaN,Replace LCD,N5V22316H-391L,LG,DOA units inspection
83479,2022-06-18,14TD90P-G.AX7GK,201QCJZ579084,NG,NaT,D-Part 돌출,NaN,Repair MB,N5V22316H-391L,LG,DOA units inspection
83489,2022-06-18,14TD90P-G.AX7GK,201QCYQ580146,NG,NaT,D-Part 돌출,NaN,Replace C cover,N5V22316H-391L,LG,DOA units inspection


In [326]:
df.loc[cond1&cond4, 'Defect Reason_English'] = df.loc[cond1&cond4, 'Defect Reason_Korean'].apply(get_translate, args=['en'])

In [327]:
df.loc[cond1&cond4]

,Inspection Date,Mapping Model.Suffix,S/N,Judge,MFG DATE,Defect Reason_Korean,Defect Reason_English,Repair Report,RMA Number,Responsibility,Inspection_Reason
83454,2022-06-18,14T90P-G.AAF6K,112QCZP588845,NG,NaT,LCD Open/Close Noise,LCD Open/Close Noise,Repair MB,N5V22316H-391L,LG,DOA units inspection
83456,2022-06-18,14T90P-G.AR5BK,201QCMR555915,NG,NaT,D-Part Dent,D-Part Dent,Replace C cover,N5V22316H-391L,LG,DOA units inspection
83460,2022-06-18,14T90P-G.AR5GK,112QCWC587345,NG,NaT,"D-Part 돌출, Box 불량","D-part protrusion, box defective",Replace T/P,N5V22316H-391L,LG,DOA units inspection
83461,2022-06-18,14T90P-G.AR5GK,112QCYQ583994,NG,NaT,D-Part 소음,D-Part Noise,Replace B cover,N5V22316H-391L,LG,DOA units inspection
83470,2022-06-18,14T90P-G.AR5GK,202QCRN561094,NG,NaT,B-Part Scratch/Dent,B-Part Scratch/Dent,Replace LCD,N5V22316H-391L,LG,DOA units inspection
83479,2022-06-18,14TD90P-G.AX7GK,201QCJZ579084,NG,NaT,D-Part 돌출,D-Part protrusion,Repair MB,N5V22316H-391L,LG,DOA units inspection
83489,2022-06-18,14TD90P-G.AX7GK,201QCYQ580146,NG,NaT,D-Part 돌출,D-Part protrusion,Replace C cover,N5V22316H-391L,LG,DOA units inspection


In [ ]:
df['Inspection Date'] = pd.to_datetime(df['Inspection Date'])

In [328]:
with open('D:/Data/PC Inspection DB.bin', 'wb') as f:
    pickle.dump(df, f)